Change the words between the apostrophies to the file name for the paylocity report and the desired name of the generated report.  Be sure to include the file ending (.txt, .csv, .xslx) for the input CSV.  Be sure that the input file is in the same folder as the script's folder.  After these changes have been made, click Run All and the program should create a new csv file in the folder the script is located in.  This file should be able to be opened in Excel.

In [3]:
#Variable inputs here!  paylocity_report_csv is the initial file and output_report_file_name is the desired name of the file generated from the program.
paylocity_report_csv = 'Insurance Scrubbed.csv'
output_report_file_name = 'Scrubbed'
#Alter nothing outside of this kernel.

The below code block imports libraries that are essential for this type of data manipulation.  Pandas is a library for dataframes (fancy tables) and NumPy is the mathematic backbone of any calculations done on dataframes.  Openpyxl allows for Excel manipulation in Python, Datetime is a library for calculating dates, and math is a general use library for Python.

In [4]:
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.styles import NamedStyle, Border, Side
from datetime import datetime, timedelta
import math
import numpy as np
from openpyxl.utils.dataframe import dataframe_to_rows

The below code block imports the CSV file into the script.  Be sure that the file ending is accurate to the new file (.txt, .csv, or .xlsx are the most likely file endings) and the file is in the same folder as the script's file.

In [5]:
initial_dataframe = pd.read_csv(paylocity_report_csv)
format = "%m/%d/%Y"
initial_time = datetime.now()

The below block of code takes only the rows that have Vendor Name as Anthem Life and makes a new dataframe from it.  A new column of the combined first and last name in the format [Last Name, First Name] is then added for readability.

In [6]:
anthemLife_dataframe = initial_dataframe.loc[initial_dataframe['Vendor Name'] == 'Anthem Life']
anthemLife_dataframe = anthemLife_dataframe.sort_values('Last Name')
anthemLife_dataframe['Employee Name'] = anthemLife_dataframe['Covered Last Name'] + ', ' + anthemLife_dataframe['Covered First Name']
pref_name_to_real_name_dict = {}
for index,row in anthemLife_dataframe.iterrows():
    pref_name_to_real_name_dict[row['Last Name'] + ', ' + row['Preferred/First Name']] = row['Employee Name']

The below block of code creates a pivot table where the index is the employee and the columns are spouse/children where the values are the total number of spouse/children.

In [7]:
spouse_child_dataframe = initial_dataframe.loc[initial_dataframe['Covered Relationship'] != 'Employee']
spouse_child_dataframe['Preferred Name'] = spouse_child_dataframe['Last Name'] + ', ' + spouse_child_dataframe['Preferred/First Name']
spouse_child_dataframe['Employee Name'] = None
for index, row in spouse_child_dataframe.iterrows():
    spouse_child_dataframe.at[index, 'Employee Name'] = pref_name_to_real_name_dict[row['Preferred Name']]
pivot_spouse_child_frame = pd.pivot_table(data = spouse_child_dataframe, values = 'Covered First Name', index = 'Employee Name', columns = 'Covered Relationship', aggfunc = pd.Series.nunique).fillna(0)

C:\Users\Owen\AppData\Local\Temp\ipykernel_9972\3103713607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spouse_child_dataframe['Preferred Name'] = spouse_child_dataframe['Last Name'] + ', ' + spouse_child_dataframe['Preferred/First Name']
C:\Users\Owen\AppData\Local\Temp\ipykernel_9972\3103713607.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  spouse_child_dataframe['Employee Name'] = None


The below block of code creates a dictionary where the keys are employee names and the values are a 1 or 0 depending on if they are an executive or not.

In [8]:
exec_dict = {}
executive_dummy_dataframe = pd.get_dummies(data = anthemLife_dataframe, columns = ['Benefit Class'])
for index, row in executive_dummy_dataframe.iterrows():
    if row['Benefit Class_EXECUTIVES'] == 1:
        exec_dict[row['Employee Name']] = 1
    else:
        exec_dict[row['Employee Name']] = 0
exec_count = 0
for val in exec_dict.values():
    if val == 1:
        exec_count += 1
del executive_dummy_dataframe

The below block of code creates a dictionary where the keys are employee names and the values are their current age.  A helper function is also made to count ages in ranges of an iterable (a list of employee names) and returns two dictionaries, where one is the counts of the age ranges, and one is a list of all the employees who fall into those age ranges.

In [9]:
employee_age_dict = {}
spouse_age_dict = {}
#for index, row in spouse_child_dataframe.loc[spouse_child_dataframe['Covered Relationship'] == 'Spouse'].iterrows():
#    birth_day = datetime.strptime(row['Covered Date of Birth'], format)
#    timedelta = initial_time - birth_day
#    spouse_age_dict[row['Employee Name']] = math.floor(timedelta.days/365)
for index, row in anthemLife_dataframe.iterrows():
    birth_day = datetime.strptime(row['Birth Date'], format)
    timedelta = initial_time - birth_day
    employee_age_dict[row['Employee Name']] = math.floor(timedelta.days/365)

for key in employee_age_dict.keys():
    spouse_age_dict[key] = employee_age_dict[key]

def count_ages(iterable, employee_age_dict):
    age_count_dict = {'0-24':0, '25-29':0,'30-34':0,'35-39':0,'40-44':0,'45-49':0,'50-54':0,'55-59':0,'60-64':0,'65-69':0}
    age_range_dict = {'0-24':[], '25-29':[],'30-34':[],'35-39':[],'40-44':[],'45-49':[],'50-54':[],'55-59':[],'60-64':[],'65-69':[]}
    for key in iterable:
        if employee_age_dict[key] < 25:
            age_count_dict['0-24'] += 1
            age_range_dict['0-24'].append(key)
        elif employee_age_dict[key] < 30:
            age_count_dict['25-29'] += 1
            age_range_dict['25-29'].append(key)
        elif employee_age_dict[key] < 35:
            age_count_dict['30-34'] += 1
            age_range_dict['30-34'].append(key)
        elif employee_age_dict[key] < 40:
            age_count_dict['35-39'] += 1
            age_range_dict['35-39'].append(key)
        elif employee_age_dict[key] < 45:
            age_count_dict['40-44'] += 1
            age_range_dict['40-44'].append(key)
        elif employee_age_dict[key] < 50:
            age_count_dict['45-49'] += 1
            age_range_dict['45-49'].append(key)
        elif employee_age_dict[key] < 55:
            age_count_dict['50-54'] += 1
            age_range_dict['50-54'].append(key)
        elif employee_age_dict[key] < 60:
            age_count_dict['55-59'] += 1
            age_range_dict['55-59'].append(key)
        elif employee_age_dict[key] < 65:
            age_count_dict['60-64'] += 1
            age_range_dict['60-64'].append(key)
        elif employee_age_dict[key] < 70:
            age_count_dict['65-69'] += 1
            age_range_dict['65-69'].append(key)
    return age_count_dict, age_range_dict

The below block of code creates two dataframes from the anthemLife_dataframe by pivoting the type of coverage to the column and the rates/coverages to the values.  These two dataframes are then joined on Employee name.  Flat Rate 1 Column is dropped from the dataframe and data without entries is replaced with a 0 as opposed to being N/A for calculation purposes.

In [10]:
final_columns = ['Employee Name','Long-Term Disability Coverage', 'Long-term Disability Rate', 'Short-Term Disability Coverage', 'Short-Term Disability Rate', 'Basic Life Coverage', 'Basic Life Rate','Dependent Life Coverage','Dependent Life Rate' ,'Voluntary Life Coverage', 'Voluntary Life Rate', 'Voluntary AD&D Coverage', 'Voluntary AD&D Rate', 'Voluntary Dependent Life Coverage', 'Voluntary Dependent Life Rate', 'Voluntary Dependent AD&D Coverage', 'Voluntary Dependent AD&D Rate', 'Total Rate']

total_coverage_pivot_frame = pd.pivot(data = anthemLife_dataframe, index = 'Employee Name', columns = 'Benefit Plan Type', values = 'Coverage Amount').drop(['Flat Rate 1'], axis = 'columns', errors = 'ignore')
total_coverage_pivot_frame.rename(mapper = lambda x:x + ' Coverage', axis = 'columns', inplace=True)

rate_pivot_frame = pd.pivot(data = anthemLife_dataframe, index = 'Employee Name', columns = 'Benefit Plan Type', values = 'Total Cost Per Pay Period').drop(['Flat Rate 1'], axis = 'columns', errors = 'ignore')
rate_pivot_frame.rename(mapper = lambda x:x + ' Rate', axis = 'columns', inplace=True)
rate_pivot_frame.fillna(0, inplace= True)
rate_pivot_frame['Total'] = rate_pivot_frame.sum(axis = 1)

joined_frame = total_coverage_pivot_frame.join(other = rate_pivot_frame, on = 'Employee Name')
joined_frame.fillna(0, inplace=True)
joined_frame.sort_values('Employee Name', inplace = True)

del total_coverage_pivot_frame
del rate_pivot_frame

The below block of code reorders the column names to match what the consolidated billing format is.

In [11]:
new_column_order = ['Employee Long Term Disability Insurance Coverage', 
'Employee Long Term Disability Insurance Rate',
'Employee Short Term Disability Insurance Coverage',
'Employee Short Term Disability Insurance Rate',
'Employee Basic Life Insurance Coverage',
'Employee Basic Life Insurance Rate',
'Employer Paid Dependent Life Insurance Coverage',
'Employer Paid Dependent Life Insurance Rate',
'Voluntary Employee Life Insurance Coverage',
'Voluntary Employee Life Insurance Rate',
'Voluntary Employee AD&D Insurance Coverage',
'Voluntary Employee AD&D Insurance Rate',
'Child Life Insurance Coverage',
'Child Life Insurance Rate',
'Spouse Life Insurance Coverage',
'Spouse Life Insurance Rate',
'Child AD&D Insurance Coverage',
'Child AD&D Insurance Rate',
'Spouse AD&D Insurance Coverage',
'Spouse AD&D Insurance Rate',
'Total']
joined_frame = joined_frame[new_column_order]

The below block of code loads the template workbook and saves it as a new workbook.  This new workbook then has the dataframe uploaded into a new sheet titled "Consolidated Billing Report".

In [12]:
template_workbook = load_workbook('Carrier Billing Template.xlsx')
template_workbook.save(output_report_file_name+'.xlsx')
current_workbook = load_workbook(output_report_file_name+'.xlsx')
border = NamedStyle(name = "border")
bd = Side(style = 'medium', color = '000000')
border.border = Border(left = bd, right = bd, top = bd, bottom = bd)

Importing data into spreadsheet below

In [13]:
ws = current_workbook['Carrier Billing']
#Uploading Long Term Disability counts for Execs and Non Execs
ws['E7'] = exec_count

exec_total_ben = 0
non_exec_total_ben = 0
exec_total_rate = 0
non_exec_total_rate = 0
for key, val in  exec_dict.items():
    if val == 1:
        exec_total_ben += joined_frame['Employee Long Term Disability Insurance Coverage'][key]
        exec_total_rate += joined_frame['Employee Long Term Disability Insurance Rate'][key]
    else:
        non_exec_total_ben += joined_frame['Employee Long Term Disability Insurance Coverage'][key]
        non_exec_total_rate += joined_frame['Employee Long Term Disability Insurance Rate'][key]
ws['G7'] = exec_total_ben

ws['E11'] = len(joined_frame) - exec_count

ws['G11'] = non_exec_total_ben

ws['M7'] = exec_total_rate

ws['M11'] = non_exec_total_rate
#Uploading Short Disability Counts and totals
ws['E16'] = len(joined_frame)

ws['G16'] = np.sum(joined_frame['Employee Short Term Disability Insurance Coverage'])

ws['M16'] = np.sum(joined_frame['Employee Short Term Disability Insurance Rate'])
#Uploading Basic Life Insurance Counts and Totals
ws['E21'] = len(joined_frame)

ws['G21'] = np.sum(joined_frame['Employee Basic Life Insurance Coverage'])

ws['M21'] = np.sum(joined_frame['Employee Basic Life Insurance Rate'])
#Uploading Voluntary Life Counts and Totals while sorting by age
volun_life_ages, volun_life_names = count_ages(joined_frame.loc[joined_frame['Voluntary Employee Life Insurance Coverage'] != 0].index, employee_age_dict)
for index, count in enumerate(volun_life_ages.values()):
    ws['E'+str(32+index)] = count
for index, names in enumerate(volun_life_names.values()):
    total_ben = 0
    total_rate = 0
    for name in names:
        total_ben += joined_frame['Voluntary Employee Life Insurance Coverage'][name]
        total_rate += joined_frame['Voluntary Employee Life Insurance Rate'][name]
    ws['G'+str(32+index)] = total_ben
    ws['M'+str(32+index)] = total_rate
#Uploading Voluntary AD&D Counts and Totals while sorting by age
volun_add_ages, volun_add_names = count_ages(joined_frame.loc[joined_frame['Voluntary Employee AD&D Insurance Coverage'] != 0].index, employee_age_dict)
for index, count in enumerate(volun_add_ages.values()):
    ws['E'+str(47+index)] = count
for index, names in enumerate(volun_add_names.values()):
    total_ben = 0
    total_rate = 0
    for name in names:
        total_ben += joined_frame['Voluntary Employee AD&D Insurance Coverage'][name]
        total_rate += joined_frame['Voluntary Employee AD&D Insurance Rate'][name]
    ws['G'+str(47+index)] = total_ben
    ws['M'+str(47+index)] = total_rate
#Uploading Child Insurance count and totals while sorting by coverage amount
for index, coverage in enumerate(sorted(joined_frame['Child Life Insurance Coverage'].unique()[1:])):
    ws['E'+str(90+index)] = len(joined_frame.loc[joined_frame['Child Life Insurance Coverage'] == coverage])
    ws['G'+str(90+index)] = np.sum(joined_frame.loc[joined_frame['Child Life Insurance Coverage'] == coverage]['Child Life Insurance Coverage'])
    ws['M'+str(90+index)] = np.sum(joined_frame.loc[joined_frame['Child Life Insurance Coverage'] == coverage]['Child Life Insurance Rate'])
#Uploading Child AD&D insurance count and totals while sorting by coverage amount
for index, coverage in enumerate(sorted(joined_frame['Child AD&D Insurance Coverage'].unique()[1:])):
    ws['E'+str(98+index)] = len(joined_frame.loc[joined_frame['Child AD&D Insurance Coverage'] == coverage])
    ws['G'+str(98+index)] = np.sum(joined_frame.loc[joined_frame['Child AD&D Insurance Coverage'] == coverage]['Child AD&D Insurance Coverage'])
    ws['M'+str(98+index)] = np.sum(joined_frame.loc[joined_frame['Child AD&D Insurance Coverage'] == coverage]['Child AD&D Insurance Rate'])
#Uploading the amount of dependents by child and spouse into employer paid dependent life insurance
ws['E26'] = np.sum(pivot_spouse_child_frame['Child'])
ws['E27'] = np.sum(pivot_spouse_child_frame['Spouse'])
ws['M26'] = np.sum(joined_frame.loc[joined_frame['Employer Paid Dependent Life Insurance Coverage'] == 2500]['Employer Paid Dependent Life Insurance Rate'])
ws['M27'] = np.sum(joined_frame.loc[joined_frame['Employer Paid Dependent Life Insurance Coverage'] == 5000]['Employer Paid Dependent Life Insurance Rate'])
#Uploading Spouse Life insurance counts and totals while sorting by age
spouse_life_ages, spouse_life_names = count_ages(joined_frame.loc[joined_frame['Spouse Life Insurance Coverage'] != 0].index, spouse_age_dict)
for index, count in enumerate(spouse_life_ages.values()):
    ws['E'+str(61+index)] = count
for index, names in enumerate(spouse_life_names.values()):
    total_ben = 0
    total_rate = 0
    for name in names:
        total_ben += joined_frame['Spouse Life Insurance Coverage'][name]
        total_rate += joined_frame['Spouse Life Insurance Rate'][name]
    ws['G'+str(61+index)] = total_ben
    ws['M'+str(61+index)] = total_rate
#Uploading spouse AD&D Insurance counts and totals while sorting by age
spouse_add_ages, spouse_add_names = count_ages(joined_frame.loc[joined_frame['Spouse AD&D Insurance Coverage'] != 0].index, spouse_age_dict)
for index, count in enumerate(spouse_add_ages.values()):
    ws['E'+str(76+index)] = count
for index, names in enumerate(spouse_add_names.values()):
    total_ben = 0
    total_rate = 0
    for name in names:
        total_ben += joined_frame['Spouse AD&D Insurance Coverage'][name]
        total_rate += joined_frame['Spouse AD&D Insurance Rate'][name]
    ws['G'+str(76+index)] = total_ben
    ws['M'+str(76+index)] = total_rate
#Saving the data uploaded above into a Excel workbook under the inputted file name

ws = current_workbook['Consolidated Billing Report']

ws['A'+str(len(joined_frame)+8)] = str(len(joined_frame)) + ' Active Employees'

alphabet = [(chr(ord('a')+i)).upper() for i in range(2,22,2)]
alphabet.append('V')

for letter in alphabet:
    ws[letter + str(len(joined_frame)+8)] = '=SUM({letter}8:{letter}{length})'.format(letter = letter, length = str(len(joined_frame)+7))

for row in ws.iter_rows(min_row=7, min_col = 0, max_row = len(joined_frame)+7, max_col = 22):
    for cell in row:
        cell.style = border

current_workbook.save(output_report_file_name+'.xlsx')

In [14]:
#Using a context manager, the data frame is uploaded into the same workbook under a seperate sheet
with pd.ExcelWriter(output_report_file_name + '.xlsx', mode = 'a', engine = 'openpyxl', if_sheet_exists='overlay') as writer:
        joined_frame.to_excel(writer, sheet_name = 'Consolidated Billing Report', startrow = 6)